# Testing Defacing PET images only

There are a few quirks to this code and this notebook is set up to help a user deal with them. For starters, the user first needs to define where each of their datasets are located in the `dataset_dictionary`, this provides common names for later comparison.

```json
dataset_dictionary = {
    "tracer1": "/full/path/to/tracer1dataset/",
    "tracer2": "/full/path/to/tracer2dataset/",
    "tracer3": "/full/path/to/tracer3dataset/"
}
```

Using the datasets above, this notebook will take the user through the defacing of each tracer using 4 different methods:

1. ~~Using the subject's MR image to register w/ and deface it's PET image.~~

2. Using `petdeface/data/sub-01/ses-baseline/anat/sub-01_T1w.nii.gz` as a template to deface each subjects PET images.

3. Using `petdeface/data/sub-mni305/anat/sub-mni305_T1w.nii.gz` ...

4. Creating a "T1" image by averaging the subjects PET image and using that instead to deface.

At the end of this pipeline the user will have ~~16~~12 different datasets.

In [22]:
dataset_dictionary = {
    "PS13": "/home/galassiae/Projects/petdeface_no_anat_data/20M0157_MDD_COX1_COX2_BIDS",
    "MC1": "/home/galassiae/Projects/petdeface_no_anat_data/COX-2_MC1_BIDS",
    "TSPO": "/home/galassiae/Projects/petdeface_no_anat_data/TSPO_Brain"
}

# we'll also need to specify were we want to keep our results

import os
import pathlib

home = pathlib.Path.home()
results_dir = home / "petdeface_jupypter_book_results"
results_dir.mkdir(exist_ok=True)


Next we build, install, and import the code we need.

In [3]:
%pip install -e .
# make sure we can import petdeface
from petdeface.petdeface import PetDeface

Obtaining file:///home/galassiae/Projects/petdeface
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for petdeface (pyproject.toml) ... done
  Created wheel for petdeface: filename=petdeface-0.2.2-py3-none-any.whl size=5842 sha256=3021cd28e454caceabd10a5b86f4d5d38b0455ea6b0f64fb7a10d71b0ff6cd7a
  Stored in directory: /home/tmp/pip-ephem-wheel-cache-_w7r4pt8/wheels/37/dd/a2/64cedb9880fd17ddc66a522d2179134857fd4f8e010f78eb55
Successfully built petdeface
  Attempting uninstall: petdeface
    Found existing installation: petdeface 0.2.2
    Uninstalling petdeface-0.2.2:
      Successfully uninstalled petdeface-0.2.2

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# We're going to make a mess if we don't use tempfile
from tempfile import TemporaryDirectory
import shutil
import os

# now let's get to defacing using a real t1w, but not the real t1w for each subject.
for tracer, dataset in dataset_dictionary.items():
    tmpdir = TemporaryDirectory()
    faced_dir = pathlib.Path(os.path.join(tmpdir.name, tracer))
    shutil.copytree(dataset, faced_dir, dirs_exist_ok=True)
    petdeface = PetDeface(
        bids_dir=faced_dir,
        output_dir=faced_dir,
        anat_only=False,
        n_procs=4,
        skip_bids_validator=True,
        remove_existing=False,
        placement="adjacent",
        preview_pics=False,
        use_template_anat="t1"
        )
    # let's check to see that the data is in the right place
    print(sorted(os.listdir(faced_dir)))

    # now we run petdeface on each dataset

    petdeface.run()

    # then once it's finished running we copy over the defaced data to our results directory
    shutil.copytree(os.path.join(tmpdir, tracer + "_defaced"), results_dir / tracer + "_defaced")

['.DS_Store', '._.DS_Store', '.bidsignore', '.fr-nc4Qtw', '.fr-ogygl3', 'dataset_description.json', 'derivatives', 'participants.json', 'participants.tsv', 'sub-01', 'sub-02', 'sub-03']
Copied t1 anatomical image to /home/tmp/tmp43pybwbh/PS13/sub-01/anat/sub-01_T1w.nii.gz
Copied t1 anatomical metadata to /home/tmp/tmp43pybwbh/PS13/sub-01/anat/sub-01_T1w.json
Copied t1 anatomical image to /home/tmp/tmp43pybwbh/PS13/sub-02/anat/sub-02_T1w.nii.gz
Copied t1 anatomical metadata to /home/tmp/tmp43pybwbh/PS13/sub-02/anat/sub-02_T1w.json
Copied t1 anatomical image to /home/tmp/tmp43pybwbh/PS13/sub-03/anat/sub-03_T1w.nii.gz
Copied t1 anatomical metadata to /home/tmp/tmp43pybwbh/PS13/sub-03/anat/sub-03_T1w.json
250425-16:28:06,113 nipype.workflow INFO:
	 Generated workflow graph: /home/tmp/tmp43pybwbh/PS13/petdeface_wf/petdeface.png (graph2use=colored, simple_form=True).
250425-16:28:06,127 nipype.workflow INFO:
	 Workflow petdeface_wf settings: ['check', 'execution', 'logging', 'monitoring']
25